In [1]:
from dfs_service import DFS, distance_obj,filter_dups,find_best_connection_distance,MySolutionMap
from dfs_service import connection_exists,validate_connection, calculate_path_distance
from itertools import permutations
import settings
import time
from decimal import Decimal
from utils import load_maps
from collections import defaultdict

In [2]:
distances = distance_obj()
ride_map = load_maps()

In [3]:
ride_map

{64430: OrderedDict([('id', '64430'),
              ('gearth_id', '0005A7BFDB13B09F848E'),
              ('name', "gadget's go coaster"),
              ('lat', Decimal('33.8152241')),
              ('long', Decimal('-117.9193603')),
              ('region', 'TOON_TOWN')]),
 58559: OrderedDict([('id', '58559'),
              ('gearth_id', '005DCC8A8413B6B56EFF'),
              ('name', 'rail road - tomorrow land'),
              ('lat', Decimal('33.81236010577997')),
              ('long', Decimal('-117.9162380822138')),
              ('region', 'TOMORROW_LAND')]),
 76868: OrderedDict([('id', '76868'),
              ('gearth_id', '007D6F8B3313A7A71F44'),
              ('name', 'indiana jones™ adventure'),
              ('lat', Decimal('33.8111657')),
              ('long', Decimal('-117.9205289')),
              ('region', 'ADVENTURE_LAND')]),
 280: OrderedDict([('id', '280'),
              ('gearth_id', '00C399D54913C25A1A98'),
              ('name', 'millennium falcon: smugglers run')

In [4]:
 # nested loop,region to region connection
all_regions_connections = permutations([region_name for region_list, region_name in settings.REGIONS_LIST])
all_regions_connections = list(all_regions_connections)
print(f" permutations of all regions {len(list(all_regions_connections))}\n")

 permutations of all regions 362880



In [5]:
# ride_map

In [6]:
solution_map = MySolutionMap()



t0 = time.time()
# for region_list, region_name in (settings.REGIONS_LIST +     [(settings.FANTASY_LAND | settings.TOON_TOWN, "FANTASY_TOON")] ):
for region_list, region_name in (settings.REGIONS_LIST  ):
    
    t1 = time.time()
    d = DFS(region_list)
    solutions = d.start_new()
    filtered_solutions = filter_dups(solutions, distances)
    # solution_map[region_name] = d.start()
    solution_map[region_name] = filtered_solutions

    print(
        f" {region_name} region took {time.time() - t1}, total solutions {len(solutions)} after filter {len(filtered_solutions)}\n")

print(f" full searches took {time.time() - t0}")



            

took 1.5735626220703125e-05 
 STAR_WARS_GALAXY_EDGE region took 0.003710031509399414, total solutions 2 after filter 2

took 0.017960309982299805 
 TOON_TOWN region took 0.026123046875, total solutions 1728 after filter 54

took 2.6226043701171875e-05 
 CRITTER_COUNTRY region took 0.002830028533935547, total solutions 6 after filter 6

took 7.200241088867188e-05 
 NEW_ORLEANS_SQUARE region took 0.0026848316192626953, total solutions 4 after filter 4

took 0.0001087188720703125 
 FRONTIER_LAND region took 0.0026938915252685547, total solutions 4 after filter 4

took 4.291534423828125e-05 
 ADVENTURE_LAND region took 0.0025370121002197266, total solutions 4 after filter 4

took 0.004349946975708008 
 MAIN_STREET region took 0.008376121520996094, total solutions 648 after filter 42

took 0.0027840137481689453 
 TOMORROW_LAND region took 0.005515098571777344, total solutions 72 after filter 24

took 66.57667279243469 
 FANTASY_LAND region took 71.74733424186707, total solutions 1276512 aft

In [7]:
#making sure path exists backwards too
oopsie = 0
iterations = 0

tmp = {}
for r in solution_map:
    
    
    for k1,k2 in solution_map[r]:
        
        if (k2,k1) not in solution_map[r]:
            oopsie+=1
#             solution_map[r][ (k2,k1)  ] = list(reversed(solution_map[r][( k1,k2)  ]))
            tmp[ (k2,k1)  ] = list(reversed(solution_map[r][( k1,k2)  ]))
    
        iterations +=1

solution_map[r].update(tmp)
        
        
        

print(oopsie, iterations)

0 184


In [8]:
solution_map["NEW_ORLEANS_SQUARE"]

{(69026, 19797): [69026, 6984, 76976, 72179, 19797],
 (69026, 72179): [69026, 6984, 76976, 19797, 72179],
 (72179, 69026): [72179, 19797, 76976, 6984, 69026],
 (19797, 69026): [19797, 72179, 76976, 6984, 69026]}

In [17]:
# permutation of all region connects
# we need to start in main street, so no reason to generate permutations where main street is not first
perms = permutations([name for _, name in settings.REGIONS_LIST if name not in { settings.MAIN_STREET_NAME, settings.TOON_TOWN_NAME}])
perms = list(perms)
print(len(perms))
print(f"all region names {[name for _, name in settings.REGIONS_LIST if name not in { settings.MAIN_STREET_NAME, settings.TOON_TOWN_NAME}]}")
perms = [[settings.MAIN_STREET_NAME] + list(p) for p in perms]
print(len(perms))

print(f"found {len(perms)} region to region paths")
print(f"path contains {len(perms[0]) } elements")

5040
all region names ['STAR_WARS_GALAXY_EDGE', 'CRITTER_COUNTRY', 'NEW_ORLEANS_SQUARE', 'FRONTIER_LAND', 'ADVENTURE_LAND', 'TOMORROW_LAND', 'FANTASY_LAND']
5040
found 5040 region to region paths
path contains 8 elements


In [18]:
perms2 = [name for _, name in settings.REGIONS_LIST if name not in { settings.MAIN_STREET_NAME, settings.TOON_TOWN_NAME, settings.FANTASY_LAND_NAME}]

# perms2.append("FANTASY_TOON")

perms2 = permutations(perms2)

perms2 = [[settings.MAIN_STREET_NAME] + list(p) for p in perms2]

print(len(perms2))



720


In [45]:
perms3 = [name for _, name in settings.REGIONS_LIST if name not in { settings.MAIN_STREET_NAME, }]


perms3 = permutations(perms3)

perms3 = [[settings.MAIN_STREET_NAME] + list(p) for p in perms3]

print(len(perms3))

40320


In [19]:
# for p in perms2:
#     if p.index("MAIN_STREET") != 0:
#         print(p)

In [20]:

# print(settings.REGIONS_LIST)

magic = ['MAIN_STREET', 'ADVENTURE_LAND', "FRONTIER_LAND", "NEW_ORLEANS_SQUARE", "CRITTER_COUNTRY", "STAR_WARS_GALAXY_EDGE", "FANTASY_LAND"]
for idx, p in enumerate(perms):
    
    
    if (magic[0] == p[0]) and (magic[1] == p[1]) and (magic[2] == p[2]) and (magic[3] == p[3]):
        if (magic[4] == p[4]) and (magic[5] == p[5]) and (magic[6] == p[6]):
            print(p, idx)
        
        
        
    

['MAIN_STREET', 'ADVENTURE_LAND', 'FRONTIER_LAND', 'NEW_ORLEANS_SQUARE', 'CRITTER_COUNTRY', 'STAR_WARS_GALAXY_EDGE', 'FANTASY_LAND', 'TOMORROW_LAND'] 3295


In [21]:
# starting in main street,
# inside each solution for source region,check exit node
# check if exit node has connection with entry node from next region
# if none found, exit path

best_distance = float("inf")
my_inf = Decimal(float("inf"))
deadend = set()
longest_path = float("-inf")
# print(len(perms))
# print(perms[0])
best_solutions = []

FLOW_SOLUTION = []
rolling_solutions = []

for pidx, path in enumerate(perms):
    
    CURR_FLOW = []
    
    curr_path_distance = Decimal("0.0")
#     tmp_distance = 0
    
    #all exists from main entrance area
    source_valid = set(   [ e2 for (e1,e2) in solution_map[path[0] ]]    )
    rolling_steps = [source_valid, ]
    for i in range(1, len(path)):
        r_source = path[i - 1]
        r_target = path[i]
        
        if (r_source, r_target) in deadend:
            curr_path_distance = my_inf
            
            break
        
        
#         possible_sources = [ e2 for (e1,e2) in solution_map[r_source] ]
        
        
        tmp_distance = find_best_connection_distance(r_source, r_target, solution_map)
        source_valid = validate_connection(r_source, r_target, solution_map, source_valid)
       
        if len(source_valid) == 0:
            print(f"no VALID connection between {r_source} {r_target}, {i}th in path {pidx}")
            deadend.add((r_source, r_target))
            break
        
        
        rolling_steps.append(source_valid)
        
        curr_path_distance += tmp_distance
        
        if tmp_distance == my_inf:
            print(f"no connection between {r_source} {r_target}, {i}th in path {pidx}")
            deadend.add((r_source, r_target))
            if i >= 7:
                print(f"{path}")
            
            print("\n")
            longest_path = max(longest_path, i)
#             curr_path_distance = float("inf")
            break
            
#         else:

        
    
    if  curr_path_distance != my_inf  and (i == len(path)-1):
#     if  curr_path_distance != my_inf:
        best_distance = min(best_distance, curr_path_distance)
        if curr_path_distance == best_distance:
            best_path = path
            
        best_solutions.append( list(path) )
        rolling_solutions.append( list(rolling_steps ) )
        
#     if pidx == 3295:
#         print( "@@@", i , len(path)-1  ,path, curr_path_distance )

print(f"best distance path: {best_distance}")
print(f"longest reached: {longest_path}")
print(f"scanned: {pidx}")
print(f"best solutions: {len(best_solutions)}")

no VALID connection between MAIN_STREET STAR_WARS_GALAXY_EDGE, 1th in path 0
no VALID connection between MAIN_STREET CRITTER_COUNTRY, 1th in path 720
no VALID connection between MAIN_STREET NEW_ORLEANS_SQUARE, 1th in path 1440
no VALID connection between FANTASY_LAND TOMORROW_LAND, 7th in path 2161
no VALID connection between NEW_ORLEANS_SQUARE TOMORROW_LAND, 5th in path 2162
no VALID connection between NEW_ORLEANS_SQUARE FANTASY_LAND, 5th in path 2164
no VALID connection between CRITTER_COUNTRY ADVENTURE_LAND, 4th in path 2166
no VALID connection between CRITTER_COUNTRY TOMORROW_LAND, 4th in path 2172
no VALID connection between CRITTER_COUNTRY FANTASY_LAND, 4th in path 2178
no VALID connection between STAR_WARS_GALAXY_EDGE NEW_ORLEANS_SQUARE, 3th in path 2184
no VALID connection between STAR_WARS_GALAXY_EDGE ADVENTURE_LAND, 3th in path 2208
no VALID connection between STAR_WARS_GALAXY_EDGE TOMORROW_LAND, 3th in path 2232
no VALID connection between STAR_WARS_GALAXY_EDGE FANTASY_LAND,

In [22]:
best_solutions

[['MAIN_STREET',
  'FRONTIER_LAND',
  'STAR_WARS_GALAXY_EDGE',
  'CRITTER_COUNTRY',
  'NEW_ORLEANS_SQUARE',
  'ADVENTURE_LAND',
  'TOMORROW_LAND',
  'FANTASY_LAND'],
 ['MAIN_STREET',
  'FRONTIER_LAND',
  'STAR_WARS_GALAXY_EDGE',
  'CRITTER_COUNTRY',
  'NEW_ORLEANS_SQUARE',
  'ADVENTURE_LAND',
  'FANTASY_LAND',
  'TOMORROW_LAND']]

In [23]:
rolling_solutions[0]

[{40918, 50720, 60282, 70282, 82766, 83634, 91073},
 {60568},
 {49078},
 {714, 43753, 61323},
 {19797, 72179},
 {61114},
 {8115, 23238, 46507, 58559},
 {30923, 46167}]

In [46]:
#try to go over all PERMS and run DFS at the same time
bad_jumps = set([])
best_tries = defaultdict(list)

def worm_search(curr_path_idx,  path, curr_ride_list, solutions_map, winners, distance_map):
    
    if curr_path_idx >= len(path):
        
        tot = calculate_path_distance(curr_ride_list,distance_map)
#         print(f"tot {tot}, curr_ride_list len {len(curr_ride_list)}")
        winners.append(  [tot, path, curr_ride_list]  )
        return
    
    
    #get list of (entrance, exist) nodes for current area
    for (e1, e2) in solutions_map[ path[curr_path_idx] ]:
        
        #eliminate if previous node,
        k1,k2 = e1,e2
        
        if distance_map.get( (curr_ride_list[-1], e1) ,-1) == -1:
            if  distance_map.get( (curr_ride_list[-1], e2) ,-1) == -1:
                bad_jumps.add( (path[curr_path_idx-1]  , path[curr_path_idx] , curr_ride_list[-1], e1,e2 )   )
                
                best_tries[curr_path_idx].append(  path )
                
                ###
                if e1 in {70528, 60282}  and e2 in {70528, 60282}:
                    print("WTF")
                
                ###
                continue
            
            else:
                k2,k1 = e1,e2
        
        
        
        worm_search(curr_path_idx+1,  path, curr_ride_list +solutions_map[ path[curr_path_idx] ][(k1,k2)]   , solutions_map, winners, distance_map)
        
        
        

winners = []
for pidx, path in enumerate(perms3):
    #path = [tomorrowland, mainentrance ...]
    
    for (e1,e2) in solution_map["MAIN_STREET"]:
        worm_search(  1, path, solution_map["MAIN_STREET"][(e1,e2)],solution_map,winners, distances )


# print(bad_jumps)
print(f"winners! {len(winners)}")

winners! 159744


In [47]:
best_tries_printed = set()
for k in best_tries:
    print(k)
    if k >=7:
        for p in best_tries[k]:
            tt = tuple(p)
            if tt in best_tries_printed:
                continue
                
            best_tries_printed.add( tt )
            print(p)

1
3
4
5
6
7
['MAIN_STREET', 'FRONTIER_LAND', 'STAR_WARS_GALAXY_EDGE', 'CRITTER_COUNTRY', 'NEW_ORLEANS_SQUARE', 'ADVENTURE_LAND', 'TOMORROW_LAND', 'TOON_TOWN', 'FANTASY_LAND']
['MAIN_STREET', 'FRONTIER_LAND', 'STAR_WARS_GALAXY_EDGE', 'CRITTER_COUNTRY', 'NEW_ORLEANS_SQUARE', 'ADVENTURE_LAND', 'TOMORROW_LAND', 'FANTASY_LAND', 'TOON_TOWN']
['MAIN_STREET', 'FRONTIER_LAND', 'STAR_WARS_GALAXY_EDGE', 'CRITTER_COUNTRY', 'NEW_ORLEANS_SQUARE', 'ADVENTURE_LAND', 'FANTASY_LAND', 'TOON_TOWN', 'TOMORROW_LAND']
['MAIN_STREET', 'FRONTIER_LAND', 'STAR_WARS_GALAXY_EDGE', 'CRITTER_COUNTRY', 'NEW_ORLEANS_SQUARE', 'ADVENTURE_LAND', 'FANTASY_LAND', 'TOMORROW_LAND', 'TOON_TOWN']
['MAIN_STREET', 'ADVENTURE_LAND', 'NEW_ORLEANS_SQUARE', 'CRITTER_COUNTRY', 'STAR_WARS_GALAXY_EDGE', 'FANTASY_LAND', 'TOON_TOWN', 'FRONTIER_LAND', 'TOMORROW_LAND']
['MAIN_STREET', 'ADVENTURE_LAND', 'NEW_ORLEANS_SQUARE', 'CRITTER_COUNTRY', 'STAR_WARS_GALAXY_EDGE', 'FANTASY_LAND', 'TOON_TOWN', 'TOMORROW_LAND', 'FRONTIER_LAND']
8
2


In [28]:



for p in bad_jumps:
    if "FANTASY_TOON" == p[0]  and "TOMORROW_LAND" in p:
        #astro
        if 44978 in p:
            ## ride_map[64430]['name']
            print("\n")
            print("$$$", [ride_map[xx]['name'] if xx in ride_map else xx  for xx in p ])
            print("\n")
            
        else:
#             print(p)
            print([ride_map[xx]['name'] if xx in ride_map else xx  for xx in p ])



# should work:
# ('FRONTIER_LAND', 'FANTASY_TOON')
#('FANTASY_TOON', 'ADVENTURE_LAND')
#('STAR_WARS_GALAXY_EDGE', 'FANTASY_TOON')
#('FANTASY_TOON', 'MAIN_STREET')
#('FANTASY_TOON', 'TOMORROW_LAND')


In [29]:
#70528 sleeping beauty castle
#39868 roger rabbit
#99677 mickey house
#40788 minnie
#64430 gadget coaster
#60515 chip and dale tree house
#27887 donald boat
#1924 goofy house
#73844 rail road mickey town ****
#30923 small world

# for k in solution_map["FANTASY_TOON"]:
#     print(ride_map[k[0]]['name'], "--",ride_map[k[1]]['name'])
    

print(" \n\n")
for k in solution_map["TOMORROW_LAND"]:
    print(ride_map[k[0]]['name'], "--",ride_map[k[1]]['name'])
    


 


buzz lightyear astro blasters -- rail road - tomorrow land
buzz lightyear astro blasters -- monorail
buzz lightyear astro blasters -- autopia
buzz lightyear astro blasters -- finding nemo submarine voyage
finding nemo submarine voyage -- space mountain
finding nemo submarine voyage -- astro orbitor
finding nemo submarine voyage -- buzz lightyear astro blasters
autopia -- space mountain
autopia -- astro orbitor
autopia -- buzz lightyear astro blasters
astro orbitor -- rail road - tomorrow land
astro orbitor -- monorail
astro orbitor -- autopia
astro orbitor -- finding nemo submarine voyage
monorail -- space mountain
monorail -- astro orbitor
monorail -- buzz lightyear astro blasters
space mountain -- rail road - tomorrow land
space mountain -- monorail
space mountain -- autopia
space mountain -- finding nemo submarine voyage
rail road - tomorrow land -- space mountain
rail road - tomorrow land -- astro orbitor
rail road - tomorrow land -- buzz lightyear astro blasters


In [30]:
#50720 disney gallery
#91073 rail road entrance
#70282 main street vehicles
#82766 lincoln
#83634 pumpkin
#40918 cinema
#60282 fortune teller mainstreet



myset= set()
for k in solution_map["MAIN_STREET"]:
    print(k)

# print(myset)

(50720, 60282)
(50720, 40918)
(50720, 83634)
(50720, 82766)
(50720, 70282)
(50720, 91073)
(91073, 60282)
(91073, 40918)
(91073, 83634)
(91073, 82766)
(91073, 70282)
(91073, 50720)
(70282, 60282)
(70282, 40918)
(70282, 83634)
(70282, 91073)
(70282, 82766)
(70282, 50720)
(82766, 60282)
(82766, 40918)
(82766, 83634)
(82766, 91073)
(82766, 70282)
(82766, 50720)
(83634, 60282)
(83634, 82766)
(83634, 40918)
(83634, 70282)
(83634, 91073)
(83634, 50720)
(40918, 60282)
(40918, 82766)
(40918, 83634)
(40918, 70282)
(40918, 91073)
(40918, 50720)
(60282, 40918)
(60282, 83634)
(60282, 91073)
(60282, 82766)
(60282, 50720)
(60282, 70282)


In [31]:
#68863 buzz
#23238 nemo
#46507 autopia
#44978   astro orbitor
#8115  monorail
#45812 space mountain
#58559 rail road tomorrow




myset= set()
for k in solution_map["TOMORROW_LAND"]:
    myset.add(k[0])
    myset.add(k[1])

print(myset)

{68863, 23238, 46507, 44978, 8115, 45812, 58559}


In [48]:
print(len(winners))
print(winners[0][0])
winners.sort(key=lambda x: x[0])
# # distances
print(winners[0][1])

print(winners[-1][0])
print(winners[-1][1])


test = {  tuple(p[1])  for p in winners  }
print(len(test))


159744
3.404101765818839796045
['MAIN_STREET', 'FRONTIER_LAND', 'STAR_WARS_GALAXY_EDGE', 'CRITTER_COUNTRY', 'NEW_ORLEANS_SQUARE', 'ADVENTURE_LAND', 'TOMORROW_LAND', 'FANTASY_LAND', 'TOON_TOWN']
3.550528601789760801045
['MAIN_STREET', 'FRONTIER_LAND', 'STAR_WARS_GALAXY_EDGE', 'CRITTER_COUNTRY', 'NEW_ORLEANS_SQUARE', 'ADVENTURE_LAND', 'TOMORROW_LAND', 'FANTASY_LAND', 'TOON_TOWN']
1


In [51]:
for n in winners[0][2]:
    print(ride_map[n]['name'])

rail road - entrance
disney gallery
great moments with mr. lincoln
main street vehicles
pumpkin festival
main street cinema
fortune teller - main street
shooting exposition
sailing columbia
pirate's lair on tom sawyer island
mark twain riverboat
thunder rail road
millennium falcon: smugglers run
rise of the resistance
davy crockett's explorer canoes
the many adventures of winnie the pooh
splash mountain
haunted mansion
haunted mansion holiday
rail road - new orleans
fortune teller - new orleans
pirates of the caribbean
indiana jones™ adventure
tarzan's tree house
jungle cruise
walt disney's enchanted tiki room
astro orbitor
buzz lightyear astro blasters
star tours – the adventures continue
space mountain
star wars launch bay
rail road - tomorrow land
autopia
monorail
finding nemo submarine voyage
matterhorn bobsleds
alice in wonderland
peter pan's flight
sleeping beauty castle walkthrough
snow white's scary adventures
pinocchio's daring journey
casey's jr train
dumbo the flying elephan

In [33]:
solution_map['TOON_TOWN']

{(60515, 39868): [60515, 64430, 99677, 40788, 1924, 27887, 73844, 39868],
 (60515, 73844): [60515, 64430, 99677, 40788, 1924, 27887, 39868, 73844],
 (60515, 99677): [60515, 64430, 1924, 39868, 73844, 27887, 40788, 99677],
 (60515, 40788): [60515, 64430, 27887, 73844, 39868, 1924, 99677, 40788],
 (60515, 64430): [60515, 99677, 40788, 27887, 73844, 39868, 1924, 64430],
 (60515, 27887): [60515, 64430, 99677, 40788, 1924, 39868, 73844, 27887],
 (60515, 1924): [60515, 64430, 99677, 40788, 27887, 73844, 39868, 1924],
 (1924, 39868): [1924, 64430, 60515, 99677, 40788, 27887, 73844, 39868],
 (1924, 73844): [1924, 64430, 60515, 99677, 40788, 27887, 39868, 73844],
 (1924, 99677): [1924, 39868, 73844, 27887, 40788, 64430, 60515, 99677],
 (1924, 40788): [1924, 39868, 73844, 27887, 64430, 60515, 99677, 40788],
 (1924, 64430): [1924, 39868, 73844, 27887, 40788, 99677, 60515, 64430],
 (1924, 60515): [1924, 39868, 73844, 27887, 40788, 99677, 64430, 60515],
 (64430, 99677): [64430, 60515, 1924, 39868, 

In [34]:
### use DFS to find all combinations

all_possibilities = []

#path:   main street, tomorrowland, fantasyland ....
for pidx, path in enumerate(perms):
    
        path_possibilities = [  solution_map[path[0] ]  ]
        
        for i in range(1, len(path)):
            r_source = path[i - 1]
            r_target = path[i]
            #source, target = main street, tomorrowland
            
            #is it a dead end?
            ###
            if not connection_exists(r_source, r_target, solution_map):
                continue
            
            
            
            
            #get connections:
            #source = [  [entrance, store1 ], [entrance, store2]   ]
            #target = [  [orbital, space mountain], [ buzz, nemo]   ]
            
            path_possibilities.append( solution_map[r_target] )
        
        
        if len(path) == len(path_possibilities):
            all_possibilities.append( path_possibilities  )
        
            







In [35]:
# all_possibilities[0]
# print(len(all_possibilities))
# print(len(all_possibilities[0]))
# print((all_possibilities[0]))

In [36]:
#func

def end_end_path_build(curr_path, curr_idx, path_perms ):
    
    if curr_idx >= len(curr_path):
        return True
    
    

#pick first index, check if 

In [37]:
# for posib in all_possibilities:
    
    

In [38]:
print(deadend)

{('ADVENTURE_LAND', 'CRITTER_COUNTRY'), ('NEW_ORLEANS_SQUARE', 'TOMORROW_LAND'), ('TOMORROW_LAND', 'NEW_ORLEANS_SQUARE'), ('NEW_ORLEANS_SQUARE', 'FANTASY_LAND'), ('FANTASY_LAND', 'ADVENTURE_LAND'), ('MAIN_STREET', 'CRITTER_COUNTRY'), ('TOMORROW_LAND', 'ADVENTURE_LAND'), ('FANTASY_LAND', 'NEW_ORLEANS_SQUARE'), ('FRONTIER_LAND', 'CRITTER_COUNTRY'), ('TOMORROW_LAND', 'CRITTER_COUNTRY'), ('CRITTER_COUNTRY', 'FANTASY_LAND'), ('CRITTER_COUNTRY', 'TOMORROW_LAND'), ('TOMORROW_LAND', 'STAR_WARS_GALAXY_EDGE'), ('ADVENTURE_LAND', 'FRONTIER_LAND'), ('FRONTIER_LAND', 'FANTASY_LAND'), ('ADVENTURE_LAND', 'FANTASY_LAND'), ('ADVENTURE_LAND', 'STAR_WARS_GALAXY_EDGE'), ('ADVENTURE_LAND', 'TOMORROW_LAND'), ('FANTASY_LAND', 'CRITTER_COUNTRY'), ('STAR_WARS_GALAXY_EDGE', 'FANTASY_LAND'), ('NEW_ORLEANS_SQUARE', 'STAR_WARS_GALAXY_EDGE'), ('FRONTIER_LAND', 'ADVENTURE_LAND'), ('FRONTIER_LAND', 'NEW_ORLEANS_SQUARE'), ('STAR_WARS_GALAXY_EDGE', 'ADVENTURE_LAND'), ('MAIN_STREET', 'NEW_ORLEANS_SQUARE'), ('STAR_WARS_G

In [39]:
print(len(best_solutions))
print(best_path)

2
['MAIN_STREET', 'FRONTIER_LAND', 'STAR_WARS_GALAXY_EDGE', 'CRITTER_COUNTRY', 'NEW_ORLEANS_SQUARE', 'ADVENTURE_LAND', 'FANTASY_LAND', 'TOMORROW_LAND']


In [40]:

conns = (
#     ("CRITTER_COUNTRY", "NEW_ORLEANS_SQUARE"),
#     ("ADVENTURE_LAND", "NEW_ORLEANS_SQUARE"),
#     ("ADVENTURE_LAND", "FRONTIER_LAND"),
#      ("NEW_ORLEANS_SQUARE", "FRONTIER_LAND"),
#      ("NEW_ORLEANS_SQUARE", "ADVENTURE_LAND"),
#          ("NEW_ORLEANS_SQUARE", "CRITTER_COUNTRY"),
#              ("MAIN_STREET", "ADVENTURE_LAND"),
#     ("ADVENTURE_LAND", "FRONTIER_LAND"),
#     ("FRONTIER_LAND", "NEW_ORLEANS_SQUARE"),
#     ("NEW_ORLEANS_SQUARE","CRITTER_COUNTRY",),
#     ("CRITTER_COUNTRY", "STAR_WARS_GALAXY_EDGE"),
#     ( "STAR_WARS_GALAXY_EDGE", "FANTASY_LAND"),
#     ("FANTASY_LAND", "TOMORROW_LAND"),
    
#         ("NEW_ORLEANS_SQUARE", "STAR_WARS_GALAXY_EDGE"),
#                  ("MAIN_STREET", "NEW_ORLEANS_SQUARE"),
    
    ("MAIN_STREET", "TOMORROW_LAND"),
    ("FANTASY_LAND", "TOMORROW_LAND"),
    ("FANTASY_LAND", "ADVENTURE_LAND"),
    ("FRONTIER_LAND", "ADVENTURE_LAND"),
    ("FRONTIER_LAND", "CRITTER_COUNTRY"),
    ("NEW_ORLEANS_SQUARE", "CRITTER_COUNTRY"),
    ("NEW_ORLEANS_SQUARE", "STAR_WARS_GALAXY_EDGE"),
    ("FANTASY_LAND", "TOON_TOWN"),


    
    
    


)

for r_source, r_target in conns:
    tmp_distance = find_best_connection_distance(r_source, r_target, solution_map)
    print(r_source, r_target,  tmp_distance, (r_source, r_target) in deadend )

MAIN_STREET TOMORROW_LAND 0.11178482480398058 False
FANTASY_LAND TOMORROW_LAND 0.08661118592832563 True
FANTASY_LAND ADVENTURE_LAND 0.11867273923671834 True
FRONTIER_LAND ADVENTURE_LAND 0.05174383191634434 True
FRONTIER_LAND CRITTER_COUNTRY 0.09229982679104619 True
NEW_ORLEANS_SQUARE CRITTER_COUNTRY 0.05021377438156316 False
NEW_ORLEANS_SQUARE STAR_WARS_GALAXY_EDGE Infinity True
FANTASY_LAND TOON_TOWN 0.07229029191445366 False


In [41]:
def find_best_toon_town_fantasy_link(solution_map, distances):
    # check each end node from region1 against all start node from region2
#     tmp_distance = Decimal(float("inf"))
    # get all solutions for source
    best_combo = None
#     resp_source, resp_target = None, None
    resp = Decimal("inf")
#     for sol_source in solution_map["FANTASY_LAND"]:
    for  f1  in settings.FANTASY_LAND: 
        for sol_target in solution_map["TOON_TOWN"]:
#             s1, s2 = sol_source[0], sol_source[-1]
#             t1, t2 = sol_target[0], sol_target[-1]
            t1,t2 = sol_target[0], sol_target[-1]

#             tmp_distance = min(distances.get((s1, t1), float("inf")), tmp_distance)
#             tmp_distance = min(distances.get((s1, t2), float("inf")), tmp_distance)
#             tmp_distance = min(distances.get((s2, t1), float("inf")), tmp_distance)
#             tmp_distance = min(distances.get((s2, t2), float("inf")), tmp_distance)
            tmp_distance1 = distances.get((f1, t1), Decimal("inf"))
                
            if tmp_distance1 == Decimal("inf"):
                continue
            
            tmp_distance2 = calculate_path_distance( solution_map["TOON_TOWN"][(t1,t2)]  ,distances )
            buff = Decimal('inf')
            f2_best = None
            buff = Decimal('inf')
            for f2 in  settings.FANTASY_LAND: 
                if f1 != f2:
                    continue
                    
                tmp = distances.get( (t2, f2) , Decimal("inf"))
                buff = min(buff, tmp)
                if buff == tmp:
                    f2_best = f2
            
            if buff != Decimal("inf"):
                print(tmp_distance1,tmp_distance2,buff)
                
            tmp_distance = tmp_distance1+tmp_distance2+buff
            
            resp = min(resp, tmp_distance)
            
            if resp == tmp_distance:
                best_combo = (  f1, t1,t2,f2_best )

    return (resp, best_combo)


z = find_best_toon_town_fantasy_link(solution_map, distances)

0.07266552189434301 0.298337938230608910 0.07229029191445366
0.07229029191445366 0.298337938230608910 0.07266552189434301
0.14537429855966008 0.298337938230608910 0.17774979646951314
0.17774979646951314 0.298337938230608910 0.14537429855966008


In [42]:
z

(Decimal('0.443293752039405580'), (46167, 39868, 73844, 46167))

In [43]:
for key in solution_map['FANTASY_LAND']:
    
    k1, k2 = key
    k1,k2 = ride_map[k1]['name'],ride_map[k2]['name']
    
    print(f"{(k1,k2)}: {[   ride_map[e]['name']  for e in solution_map['FANTASY_LAND'][key]    ]}\n\n")


('sleeping beauty castle walkthrough', "it's a small world"): ['sleeping beauty castle walkthrough', "peter pan's flight", "snow white's scary adventures", "pinocchio's daring journey", "casey's jr train", 'dumbo the flying elephant', 'king arthur carrousel', "mr. toad's wild ride", 'alice in wonderland', 'matterhorn bobsleds', 'mad tea party', 'storybook land canal boats', 'its a small world holiday', "it's a small world"]


('sleeping beauty castle walkthrough', 'its a small world holiday'): ['sleeping beauty castle walkthrough', "peter pan's flight", "snow white's scary adventures", "pinocchio's daring journey", "casey's jr train", 'dumbo the flying elephant', 'king arthur carrousel', "mr. toad's wild ride", 'alice in wonderland', 'matterhorn bobsleds', 'mad tea party', 'storybook land canal boats', "it's a small world", 'its a small world holiday']


('matterhorn bobsleds', "it's a small world"): ['matterhorn bobsleds', 'alice in wonderland', "peter pan's flight", 'sleeping beauty 

In [44]:
for key in solution_map['ADVENTURE_LAND']:
    
    k1, k2 = key
    k1,k2 = ride_map[k1]['name'],ride_map[k2]['name']
    
    print(f"{(k1,k2)}: {[   ride_map[e]['name']  for e in solution_map['ADVENTURE_LAND'][key]    ]}\n\n")

("walt disney's enchanted tiki room", 'indiana jones™ adventure'): ["walt disney's enchanted tiki room", 'jungle cruise', "tarzan's tree house", 'indiana jones™ adventure']


("walt disney's enchanted tiki room", "tarzan's tree house"): ["walt disney's enchanted tiki room", 'jungle cruise', 'indiana jones™ adventure', "tarzan's tree house"]


("tarzan's tree house", "walt disney's enchanted tiki room"): ["tarzan's tree house", 'indiana jones™ adventure', 'jungle cruise', "walt disney's enchanted tiki room"]


('indiana jones™ adventure', "walt disney's enchanted tiki room"): ['indiana jones™ adventure', "tarzan's tree house", 'jungle cruise', "walt disney's enchanted tiki room"]


